# Tutorial on collapsed Gibbs sampling

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

docs = [
    ['apple', 'ios', 'mac', 'book'],
    ['apple', 'mac', 'book', 'apple', 'store'],
    ['mac', 'book', 'ios', 'store'],
    ['banana', 'mango', 'fruit'],
    ['apple', 'fruit'],
    ['orange', 'strawberry', 'banana'],
    ['orange', 'mango', 'banana'],
    ['fruit', 'apple', 'mac', 'ios']
]
K = 2
words = list(set([x for y in docs for x in y]))
alpha = np.array([1]*K)
beta = np.array([1]*len(words))

## Init counters of assignments
- $n_{d,k}$ number of words of the document $d$ assigned to $k$
- $n_{k,w}$ number of times (instances of) word $w$ is assigned to $k$
- $n_k$ number of word instances assignements to $k$
- $z$ array of assignments to topics for each of the words instances

In [2]:
N = sum([len(x) for x in docs])
z = np.zeros(N)
ndk = np.zeros((len(docs), K))
nkw = np.zeros((K, len(words)))
nk = np.zeros(K)

### Randomly initialize

In [3]:
instance = 0
occurrences = []
for i, doc in enumerate(docs):
    for w in doc:
        occurrences.append(w)
        t = np.random.choice(range(0,K))
        z[instance] = t
        nk[t] += 1
        ndk[i,t] += 1
        nkw[t, words.index(w)] += 1
        instance += 1

In [4]:
display(pd.DataFrame(z, index=occurrences).T)
display(pd.DataFrame(ndk, index=range(len(docs)), columns=range(K)).T)
display(pd.DataFrame(nkw, index=range(K), columns=words))

,apple,ios,mac,book,apple,mac,book,apple,store,mac,...,orange,strawberry,banana,orange,mango,banana,fruit,apple,mac,ios
0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0


,0,1,2,3,4,5,6,7
0,2.0,2.0,1.0,0.0,0.0,3.0,0.0,1.0
1,2.0,3.0,3.0,3.0,2.0,0.0,3.0,3.0


,mango,strawberry,book,ios,orange,apple,mac,fruit,store,banana
0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,2.0,0.0,2.0,2.0,1.0,4.0,3.0,2.0,1.0,2.0


## Likelihood

In [5]:
def theta(ndk, alpha, d_i, k_i):
    return (ndk[d_i, k_i] + alpha[k_i]) / (np.sum(ndk[d_i,:]) + alpha[k_i])

def phi(nkw, beta, w_i, k_i):
    return (nkw[k_i, w_i] + beta[w_i]) / (np.sum(nkw[k_i,:] + beta))

In [6]:
print(theta(ndk, alpha, 1, 0))

0.5


# Gibbs Sampling

In [7]:
from IPython.display import clear_output

In [8]:
def copy(a):
    if a.ndim == 1:
        new_a = np.zeros(a.shape)
        for i, v in enumerate(a):
            new_a[i] = v
    else:
        new_a = np.zeros(a.shape)
        for i, row in enumerate(a):
            for j, v in enumerate(row):
                new_a[i,j] = v
    return new_a

In [9]:
def gibbs(docs, words, topics, ndk, nkw, nk, alpha, beta, iterations=10):
    history = [(copy(ndk), copy(nkw), copy(topics))]
    for it in range(0, iterations):
        w_i = 0
        for doc_i, doc in enumerate(docs):
            for w in doc:
                word = [x for y in docs for x in y][w_i]
                topic = int(topics[w_i])
                p_z = np.zeros(len(alpha))
                for k_i in range(0, len(alpha)):
                    p_z[k_i] = theta(ndk, alpha, doc_i, k_i) * phi(nkw, beta, words.index(word), k_i)
                p_z = p_z / np.sum(p_z)
                # Sample from p_z
                new_topic = np.random.choice(len(p_z), 1, p=p_z)[0]
                # Update
                # Remove current assignment
                ndk[doc_i, topic] -= 1
                nkw[topic, words.index(word)] -= 1
                nk[topic] -= 1
                topics[w_i] = new_topic
                ndk[doc_i, new_topic] += 1
                nkw[new_topic, words.index(word)] += 1
                nk[new_topic] += 1
                w_i += 1
            history.append((copy(ndk), copy(nkw), copy(topics)))
    return history

In [10]:
history = gibbs(docs, words, z, ndk, nkw, nk, alpha, beta, iterations=1000)

In [11]:
for ndk_h, nkw_h, z_h in history:
    clear_output(wait=True)
    display(pd.DataFrame(z_h, index=occurrences).T)
    display(pd.DataFrame(ndk_h, index=range(len(docs)), columns=range(K)).T)
    display(pd.DataFrame(nkw_h, index=range(K), columns=words))
    command = input()
    if command == 'quit':
        break

,apple,ios,mac,book,apple,mac,book,apple,store,mac,...,orange,strawberry,banana,orange,mango,banana,fruit,apple,mac,ios
0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


,0,1,2,3,4,5,6,7
0,1.0,0.0,2.0,0.0,0.0,3.0,1.0,0.0
1,3.0,5.0,2.0,3.0,2.0,0.0,2.0,4.0


,mango,strawberry,book,ios,orange,apple,mac,fruit,store,banana
0,0.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0
1,2.0,0.0,3.0,1.0,0.0,5.0,4.0,3.0,1.0,2.0


# Sklearn implementation

In [12]:
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter


M = np.zeros((len(words), len(docs)))
for i, doc in enumerate(docs):
    for k, w in Counter(doc).most_common():
        M[words.index(k), i] = w
LDA = LatentDirichletAllocation(n_components=2, learning_method='batch', max_iter=100).fit(M.T)

In [13]:
A = LDA.components_

In [14]:
A.T

array([[0.5046074 , 2.4953926 ],
       [0.50388532, 1.49611468],
       [3.49132847, 0.50867153],
       [3.4899988 , 0.5100012 ],
       [0.50397218, 2.49602782],
       [5.48683901, 0.51316099],
       [4.49064429, 0.50935571],
       [2.41385657, 1.58614343],
       [2.49166987, 0.50833013],
       [0.50442683, 3.49557317]])

Topic word distribution. components_[i, j] represents word j in topic i.

In [15]:
display(pd.DataFrame(A, index=range(K), columns=words))

,mango,strawberry,book,ios,orange,apple,mac,fruit,store,banana
0,0.504607,0.503885,3.491328,3.489999,0.503972,5.486839,4.490644,2.413857,2.49167,0.504427
1,2.495393,1.496115,0.508672,0.510001,2.496028,0.513161,0.509356,1.586143,0.50833,3.495573


In [16]:
Terms = np.zeros((len(words), K))
Docs = np.zeros((len(docs), K))

for w_i, w in enumerate(words):
    Terms[w_i] = A.T[w_i] / np.sum(A.T[w_i])

for d_i, d in enumerate(docs):
    dM = np.zeros((len(d), K))
    for j, w in enumerate(d):
        dM[j] = Terms[words.index(w)]
    Docs[d_i] = np.sum(dM, axis=0) / np.array([len(d)]*K)

In [17]:
display(pd.DataFrame(Terms, index=words, columns=range(0,K)))
display(pd.DataFrame(Docs, index=range(0, len(docs)), columns=range(0,K)))

,0,1
mango,0.168202,0.831798
strawberry,0.251943,0.748057
book,0.872832,0.127168
ios,0.872500,0.127500
orange,0.167991,0.832009
apple,0.914473,0.085527
mac,0.898129,0.101871
fruit,0.603464,0.396536
store,0.830557,0.169443
banana,0.126107,0.873893


,0,1
0,0.889483,0.110517
1,0.886093,0.113907
2,0.868504,0.131496
3,0.299258,0.700742
4,0.758969,0.241031
5,0.182013,0.817987
6,0.154100,0.845900
7,0.822141,0.177859


In [18]:
sigma_w, sigma_d = 4, 3
for t_k in range(0, K):
    print('topic', t_k)
    top_w = sorted([(i, wk) for i, wk in enumerate(Terms.T[t_k])], key=lambda y: -y[1])[:sigma_w]
    top_d = sorted([(i, dk) for i, dk in enumerate(Docs.T[t_k])], key=lambda y: -y[1])[:sigma_d]
    print([words[x[0]] for x in top_w])
    print([docs[x[0]] for x in top_d])
    print("")

topic 0
['apple', 'mac', 'book', 'ios']
[['apple', 'ios', 'mac', 'book'], ['apple', 'mac', 'book', 'apple', 'store'], ['mac', 'book', 'ios', 'store']]

topic 1
['banana', 'orange', 'mango', 'strawberry']
[['orange', 'mango', 'banana'], ['orange', 'strawberry', 'banana'], ['banana', 'mango', 'fruit']]

